In [1]:
import numpy as np
import pandas as pd

In [ ]:
import tensorflow as tf

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

np.set_printoptions(threshold=2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Pretrained model
tokenizer        = AutoTokenizer.from_pretrained(CHECKPOINT, local_files_only=True)
pretrained_model = TFAutoModelForSequenceClassification.from_pretrained(CHECKPOINT,
                                                                        local_files_only=True,
                                                                        from_pt=True)